# <B> Setup for SageMaker BYOM examples </B>
* Container: pytorch_p39

## AutoReload

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Set roles

In [27]:
from sagemaker import get_execution_role

In [28]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


### 2.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](images/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](images/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](images/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](images/attach-policy.png) 

In [29]:
from utils.iam import iam_handler

In [30]:
iam = iam_handler()

### 2.2 Attach policy - sagemaker execution role
> SSM  
> EC2ContainerRegistry <BR>
> S3

In [31]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

## 2. Set default parameters

In [32]:
import boto3
import sagemaker
from utils.ssm import parameter_store

In [33]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)

In [34]:
strPrefix = "SM-BYOM-"

In [35]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="".join([strPrefix, "BUCKET"]), value=sagemaker.Session().default_bucket(), overwrite=True)
pm.put_params(key="".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="".join([strPrefix, "ACCOUNT-ID"]), value=boto3.client("sts").get_caller_identity().get("Account"), overwrite=True)

'Store suceess'

## 3. Data preparation
* https://sagemaker-examples.readthedocs.io/en/latest/aws_sagemaker_studio/frameworks/pytorch_cnn_cifar10/pytorch_cnn_cifar10.html
* The CIFAR-10 dataset is a subset of the 80 million tiny images dataset. It consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class.

In [ ]:
! wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
! tar xfvz cifar-10-python.tar.gz
! mkdir data
! mv cifar-10-batches-py data/.
! rm cifar-10-python.tar.gz

--2023-03-02 02:57:42--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

100%[======================================>] 170,498,071 12.3MB/s   in 14s    

2023-03-02 02:57:57 (11.2 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5


### 3.1 preview

In [ ]:
from utils.cifar_utils import classes, show_img, train_data_loader, test_data_loader
train_loader = train_data_loader()
test_loader = test_data_loader()

In [ ]:
import numpy as np
import torchvision, torch

# get some random training images
dataiter = iter(train_loader)
images, labels = next(iter(dataiter))

# show images
show_img(torchvision.utils.make_grid(images))

# print labels
print(" ".join("%9s" % classes[labels[j]] for j in range(4)))

### 3.2 upload to S3

In [ ]:
from utils.s3 import s3_handler

In [ ]:
s3 = s3_handler()

In [ ]:
source_dir, target_bucket, target_dir = "data", pm.get_params(key=strPrefix+"BUCKET"), "data"
s3.upload_dir(source_dir, target_bucket, target_dir)

In [ ]:
pm.put_params(key="".join([strPrefix, "DATA-PATH"]), value=f"s3://{target_bucket}/{target_dir}", overwrite=True)

In [ ]:
print (f'input data: {pm.get_params(key=strPrefix + "DATA-PATH")}')